In [20]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
import hvplot.pandas
import requests

#import api key

from api_keys import geoapify_key

In [15]:
# Store filepath in a variable
OPTN_data = Path("Resources/cleaned_OPTN_data.csv")
state_data = Path("Resources/state_totals.csv")

# read the file - AP
kidney_data = pd.read_csv(OPTN_data)
state_totals = pd.read_csv(state_data)

kidney_data.head()


,State,Total,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years,Payment type,Blood type,Age,Ethnicity,Sex
0,Alabama,455,9,22,61,101,146,73,54,40,NaN,NaN,NaN,NaN,Female
1,Alabama,713,16,38,87,150,237,135,95,43,NaN,NaN,NaN,NaN,Male
2,Alabama,787,5,7,72,169,267,146,97,48,Public insurance - Medicare FFS (Fee for Service),NaN,NaN,NaN,NaN
3,Alabama,688,9,32,88,154,215,122,93,57,NaN,NaN,NaN,"Black, Non-Hispanic",NaN
4,Alabama,631,14,30,80,136,209,124,81,46,NaN,O,NaN,NaN,NaN


In [3]:
# ignoring commas

kidney_data["Total"] = kidney_data["Total"].str.replace(",", "")
kidney_data["< 30 days"] = kidney_data["< 30 days"].str.replace(",", "")
kidney_data["30 to < 90 days"] = kidney_data["30 to < 90 days"].str.replace(",", "")
kidney_data["90 days to < 6 months"] = kidney_data["90 days to < 6 months"].str.replace(",", "")
kidney_data["6 months to < 1 year"] = kidney_data["6 months to < 1 year"].str.replace(",", "")
kidney_data["1 year to < 2 years"] = kidney_data["1 year to < 2 years"].str.replace(",", "")
kidney_data["2 years to < 3 years"] = kidney_data["2 years to < 3 years"].str.replace(",", "")
kidney_data["3 years to < 5 years"] = kidney_data["3 years to < 5 years"].str.replace(",", "")
kidney_data["> 5 years"] = kidney_data["> 5 years"].str.replace(",", "")

In [4]:
# need to type total and wait time columns as integers
kidney_data["Total"] = kidney_data["Total"].astype(int)
kidney_data["< 30 days"] = kidney_data["< 30 days"].astype(int)
kidney_data["30 to < 90 days"] = kidney_data["30 to < 90 days"].astype(int)
kidney_data["90 days to < 6 months"] = kidney_data["90 days to < 6 months"].astype(int)
kidney_data["6 months to < 1 year"] = kidney_data["6 months to < 1 year"].astype(int)
kidney_data["1 year to < 2 years"] = kidney_data["1 year to < 2 years"].astype(int)
kidney_data["2 years to < 3 years"] = kidney_data["2 years to < 3 years"].astype(int)
kidney_data["3 years to < 5 years"] = kidney_data["3 years to < 5 years"].astype(int)
kidney_data["> 5 years"] = kidney_data["> 5 years"].astype(int)

In [ ]:
# testing
# variable = kidney_data.loc[(kidney_data["Sex"] == "Female") & (kidney_data["State"] == "Oklahoma"), "Total"].values[0]
# variable

### Oklahoma

In [5]:
# Total candidates for Oklahoma

ok_totals1 = kidney_data.loc[kidney_data["State"] == "Oklahoma"]
ok_male_totals = ok_totals1.loc[ok_totals1["Sex"] == "Male"]["Total"].values[0]
ok_female_totals = ok_totals1.loc[ok_totals1["Sex"] == "Female"]["Total"].values[0]
ok_totals = ok_male_totals + ok_female_totals
ok_totals

597

In [11]:
# Need to get wait time values for each blood type 

# Total for each blood type

ok_blood_type_O = ok_totals1.loc[ok_totals1["Blood type"] == "O"]["Total"].values[0]
ok_blood_type_A = ok_totals1.loc[ok_totals1["Blood type"] == "A"]["Total"].values[0]
ok_blood_type_B = ok_totals1.loc[ok_totals1["Blood type"] == "B"]["Total"].values[0]
ok_blood_type_AB = ok_totals1.loc[ok_totals1["Blood type"] == "AB"]["Total"].values[0]
print(ok_blood_type_O)
print(ok_blood_type_A)
print(ok_blood_type_B)
print(ok_blood_type_AB)

341
188
65
3


In [12]:
# Testing

#type_O_wait1 = ok_totals1.loc[ok_totals1["Blood type"] == "O"]["< 30 days"].values[0]
#type_O_wait1


16

In [29]:
# Creating blood type vs wait time dataframe

ok_blood_type_wait = pd.DataFrame(ok_totals1[["Blood type","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_blood_type_wait

,Blood type,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1067,O,16,24,39,64,82,70,48,15
1071,A,14,15,21,46,47,25,15,7
1078,B,3,9,7,13,13,8,9,5
1088,AB,0,0,1,0,1,1,0,0


In [ ]:
# Plot above on bar chart

In [30]:
# Creating sex vs wait time dataframe

ok_sex_wait = pd.DataFrame(ok_totals1[["Sex","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_sex_wait

,Sex,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1065,Female,10,17,25,45,43,39,32,11
1066,Male,23,31,43,78,100,65,40,16


In [ ]:
# Plot above on a bar chart

In [31]:
# Creating payment type vs wait time dataframe

ok_payment_wait = pd.DataFrame(ok_totals1[["Payment type","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_payment_wait

,Payment type,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1069,Private insurance,18,28,22,59,62,51,35,14
1072,Public insurance - Medicare & Choice,6,11,24,34,45,31,14,0
1076,Public insurance - Medicare FFS (Fee for Service),6,5,10,16,19,10,18,12
1081,Public insurance - Medicaid,1,3,8,10,11,8,2,0
1084,Public insurance - Department of VA,1,1,4,3,6,2,3,1
1087,Public insurance - Other government,0,0,0,1,0,2,0,0
1089,Unknown,2,0,0,0,0,0,0,0


In [ ]:
# Plot above on a bar chart

In [32]:
# Creating ethnicity vs wait time dataframe

ok_ethnicity_wait = pd.DataFrame(ok_totals1[["Ethnicity","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_ethnicity_wait

,Ethnicity,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1068,"White, Non-Hispanic",19,25,35,63,77,55,35,11
1075,"Black, Non-Hispanic",5,6,15,22,25,16,18,9
1077,"American Indian/Alaska Native, Non-Hispanic",2,5,6,15,19,12,6,3
1079,Hispanic/Latino,5,6,6,15,11,13,8,3
1082,"Asian, Non-Hispanic",1,4,3,6,8,2,2,1
1083,"Multiracial, Non-Hispanic",1,2,3,2,1,7,2,0
1086,"Pacific Islander, Non-Hispanic",0,0,0,1,2,0,1,0


In [ ]:
# Plot above on a bar chart

In [33]:
# Creating age vs wait time dataframe

ok_age_wait = pd.DataFrame(ok_totals1[["Age","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_age_wait

,Age,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1070,50-64 Years,12,23,39,47,61,40,28,10
1073,65 +,8,7,13,33,37,28,21,7
1074,35-49 Years,8,10,7,30,29,22,15,10
1080,18-34 Years,3,6,9,13,15,11,8,0
1085,1-5 Years,2,1,0,0,1,2,0,0
1090,11-17 Years,0,1,0,0,0,1,0,0


In [ ]:
# Plot above on a bar chart

In [11]:
#Print State Totals

state_totals.head()

,State,Total,< 30 Days,30 to < 90 Days,90 Days to < 6 Months,6 Months to < 1 Year,1 Year to < 2 Years,2 Years to < 3 Years,3 Years to < 5 Years,5 or More Years
0,Alabama,1166,25,60,148,251,383,208,149,83
1,Alaska,118,3,2,11,18,18,23,29,15
2,Arizona,1035,54,123,158,186,212,113,99,100
3,Arkansas,343,25,48,46,67,100,32,37,22
4,California,17409,454,893,1154,2188,3476,2463,3779,3699


In [27]:
#Add columns to the state Totals for Lat and Lng
state_totals ["Lat"] = ""
state_totals["Lon"] = ""
state_totals.head()

,State,Total,< 30 Days,30 to < 90 Days,90 Days to < 6 Months,6 Months to < 1 Year,1 Year to < 2 Years,2 Years to < 3 Years,3 Years to < 5 Years,5 or More Years,Lat,Lon
0,Alabama,1166,25,60,148,251,383,208,149,83,,
1,Alaska,118,3,2,11,18,18,23,29,15,,
2,Arizona,1035,54,123,158,186,212,113,99,100,,
3,Arkansas,343,25,48,46,67,100,32,37,22,,
4,California,17409,454,893,1154,2188,3476,2463,3779,3699,,


In [28]:
#Set search parameters
params = {
        "apiKey" : geoapify_key,
        "format":"json"
}
base_url = "https://api.geoapify.com/v1/geocode/search"



In [29]:
# Print a message to follow up the airport search
print("Starting state search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in state_totals.iterrows():

    # Get the state's name & add ", United States of America" to the string so geoapify finds the correct city
    state = row["State"] + ", United States of America"

    # Add the current city to the parameters
    params["text"] = f"{state}"

    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON
    response = response.json()

    # Extract latitude and longitude
    state_totals.loc[index, "Lat"] = response["results"][0]["lat"]
    state_totals.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {state} fetched...")

# Display sample data to confirm that the coordinates appear
state_totals.head()

Starting state search
Coordinates for Alabama, United States of America fetched...
Coordinates for Alaska, United States of America fetched...
Coordinates for Arizona, United States of America fetched...
Coordinates for Arkansas, United States of America fetched...
Coordinates for California, United States of America fetched...
Coordinates for Colorado, United States of America fetched...
Coordinates for Connecticut, United States of America fetched...
Coordinates for Delaware, United States of America fetched...
Coordinates for District of Columbia, United States of America fetched...
Coordinates for Florida, United States of America fetched...
Coordinates for Georgia, United States of America fetched...
Coordinates for Hawaii, United States of America fetched...
Coordinates for Idaho, United States of America fetched...
Coordinates for Illinois, United States of America fetched...
Coordinates for Indiana, United States of America fetched...
Coordinates for Iowa, United States of Amer

,State,Total,< 30 Days,30 to < 90 Days,90 Days to < 6 Months,6 Months to < 1 Year,1 Year to < 2 Years,2 Years to < 3 Years,3 Years to < 5 Years,5 or More Years,Lat,Lon
0,Alabama,1166,25,60,148,251,383,208,149,83,33.258882,-86.829534
1,Alaska,118,3,2,11,18,18,23,29,15,64.445961,-149.680909
2,Arizona,1035,54,123,158,186,212,113,99,100,34.395342,-111.763275
3,Arkansas,343,25,48,46,67,100,32,37,22,35.204888,-92.447911
4,California,17409,454,893,1154,2188,3476,2463,3779,3699,36.701463,-118.755997


In [39]:
%%capture --no-display

# Configure the map plot
state_map = state_totals.hvplot.points( "Lon",
                                    "Lat",
                                    geo = True,
                                    color = "State",
                                    tiles = "OSM",
                                    frame_width = 800,
                                    frame_height = 600,
                                    size = "Total",
                                    scale = 0.3,
                                    hover_cols = ["State", "Total"]
                                    )

# Display the map
state_map

#Save the image
#plt.savefig("/Resources/state_map.png")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (State,Total)